[Reference](https://eljand.medium.com/parallelize-functions-and-ml-models-in-python-dafe67f6e5d5)

In [1]:
!pip install -U ray #if you are using Jupyter notebook

     |████████████████████████████████| 51.5 MB 23 kB/s 
     |████████████████████████████████| 72 kB 633 kB/s 
     |████████████████████████████████| 127 kB 59.3 MB/s 
     |████████████████████████████████| 65 kB 4.4 MB/s 
     |████████████████████████████████| 1.3 MB 36.4 MB/s 
     |████████████████████████████████| 78 kB 7.4 MB/s 
     |████████████████████████████████| 3.1 MB 39.2 MB/s 
     |████████████████████████████████| 10.1 MB 39.3 MB/s 
     |████████████████████████████████| 294 kB 69.5 MB/s 
     |████████████████████████████████| 142 kB 45.6 MB/s 
     |████████████████████████████████| 85 kB 4.2 MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-py3-none-any.whl size=12617 sha256=f71fd11bef113562701636e78ea32c4199225ba9b0ba7a314dabf280a85340a4
  Stored in directory: /root/.cache/pip/wheels/e6/67/af/f1ad15974b8fd95f59a63dbf854483ebe5c7a46a93930798b8
Successfully built gpustat


In [2]:
import time

# Function that runs the counter for 1 second
def count_per_second(x):
    start = time.time()
    counter = 0
    while time.time() - start < 1:
        counter += 1
    return counter

start = time.time()
# Execute the function 9 times
results = [count_per_second(x) for x in range(9)]
print("duration =", time.time() - start)
print("results =", results)

duration = 9.000239610671997
results = [4121668, 4510991, 4520966, 4496831, 4483132, 4586854, 4487570, 4538798, 4457891]


In [3]:
import ray
# Start ray cluster with 3 cpu's
ray.init(num_cpus=3)

# @ray.remote decorator enables to use this 
# function in distributed setting
@ray.remote
def count_per_second(x):
    start = time.time()
    counter = 0
    while time.time() - start < 1:
        counter += 1
    return counter

start = time.time()
# Run the function 9 times using multiple cores and gather the results
results = ray.get([count_per_second.remote(x) for x in range(9)])
print("duration =", time.time() - start)
print("results =", results)

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
2021-07-27 14:25:45,421	INFO services.py:1247 -- View the Ray dashboard at http://127.0.0.1:8265
(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip in

duration = 4.148217678070068
results = [1035452, 1213437, 984098, 1048291, 924094, 959036, 1405244, 1958161, 2865096]


In [4]:
!pip install transformers #if you are using Jupyter notebook

     |████████████████████████████████| 2.6 MB 29.6 MB/s 
     |████████████████████████████████| 895 kB 33.6 MB/s 
     |████████████████████████████████| 636 kB 49.0 MB/s 
     |████████████████████████████████| 3.3 MB 62.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from transformers import pipeline
# Create text generator and import distilgpt2 model
generator = pipeline('text-generation', model='distilgpt2')

# 4 different Beginnings that I use for text generation
sentence_beginnings = ["In Estonia, our main source of energy is",
                       "The main task of the technology scout is",
                       "Renewable energy is important because",
                       "Electric Vehicles are changing the"]

In [6]:
# Function that carries out model inference and return generated text
def continue_text(beginning):
    text = generator(beginning,  max_length=30, num_return_sequences=1)
    return text[0]['generated_text']
    
start = time.time()
# Generate text for all beginnings
results = [continue_text(beginning) for beginning in sentence_beginnings]
print("duration =", time.time() - start)
print(results)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


duration = 3.9638125896453857
['In Estonia, our main source of energy is fuel, which is also part of our energy sources, namely the Baltic Sea. We work in more than', 'The main task of the technology scout is to identify and identify the best and "best" products to make the job happen.\nThe process for making', 'Renewable energy is important because the potential for a new type of renewable energy generation would be significant and, as has been the case with the shale', 'Electric Vehicles are changing the way people travel and that’s not really that big anymore. In a recent interview with TNA.com, singer']


In [7]:
ray.init(num_cpus=2, ignore_reinit_error=True)

# Create remote function
@ray.remote
def continue_text(beginning):
    text = generator(beginning,  max_length=30, num_return_sequences=1)
    return text[0]['generated_text']

start = time.time()
# Send continue_text() to multiple cores and gather the results 
results = ray.get([continue_text.remote(beginning) for beginning in sentence_beginnings])
print("duration =", time.time() - start)
print(results)

2021-07-27 14:30:36,131	INFO worker.py:811 -- Calling ray.init() again after it has already been called.
2021-07-27 14:30:39,023	WARNING worker.py:1189 -- Warning: The remote function __main__.continue_text has size 335337785 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
(pid=233) 2021-07-27 14:30:47.514438: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=234) 2021-07-27 14:30:47.519848: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=297) 2021-07-27 14:30:47.898455: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=297) Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
(pid=233) Setting `pad_token_id` to `eos_token_id`:50256 for open-end generatio

duration = 20.950606107711792
['In Estonia, our main source of energy is the hydroelectric power.\n\nWhat do you think? If so, what about the local authority?', 'The main task of the technology scout is to know how much of her opponent is buying and selling when she is there. She will also have to know', "Renewable energy is important because we need it all to be energy efficient and safe. The energy cost of energy efficiency doesn't depend on the cost", 'Electric Vehicles are changing the way people have to fly. (Photos: NASA/UCLA/Getty Images)\n\n\n\n\n\n\n\n']
